# Your turn at home!

![](images/dev.gif)


## Transfer formats, XML and JSON

  * Read about marshalling and unmarshalling of XML and JSON in the second part of chapter seven in the book _Go Web Programming_, see `Go_Web_Programming_Chap7_part2.pdf`.
  * To get an introduction to XML sytnax, read chapter 1 of "XML - Visual QuickStart Guide" , see `XML_Visual_QuickStart_Guide_chap1.pdf`.

## One more protocol - HTTP

  * Read the original proposal of the _World Wide Web_ https://www.w3.org/History/1989/proposal.html together with the original implementation of HTTP, see https://www.w3.org/Protocols/HTTP/AsImplemented.html and https://www.w3.org/History/19921103-hypertext/hypertext/WWW/Protocols/HTTP.html. For the historical curious, you can read the Original WWW announcment here: https://groups.google.com/forum/#!topic/comp.sys.next.announce/avWAjISncfw

  * If you are interested in the new HTTP 2 standard you can read about it here https://www.w3.org/Protocols/HTTP/HTTP2.html and here https://en.wikipedia.org/wiki/HTTP/2. However, HTTP 2 will not be in focus of this course.
  * Read chapter 8 of _"Network Programming with Go"_, see `Network_Programming_with_Go_Chap8.pdf`.
  * Alternativley, read the introduction to HTTP in chapter 1 in _"Go Web Programming"_, see `Go_Web_Programming_Chap1_part.pdf`.

  * To develop a feeling for the commonalities and differences between a `GET` and a `POST` request, you might want to watch the following video https://www.youtube.com/watch?v=UObINRj2EGY.


## RPC
  
  * To prepare our work on remote procedure calls (RPC) read chapter 13 of _Network Programming with Go_, see `Network_Programming_with_Go_Chap13.pdf`
  * To get a visual explanation of RPCs, you may want to watch the following video again https://www.youtube.com/watch?v=gr7oaiUsxSU.
  
## RMI in Java

  * Since remote method invocations became well known with Java, read about them in chapter 18 of the book _Java Network Programming_, see `Java_Network_Programming_Chap18.pdf`.
  * Alternatively, you have the official documentaion at Oracle:
    * https://docs.oracle.com/javase/tutorial/rmi/overview.html  
    * https://docs.oracle.com/javase/tutorial/rmi/server.html
    * https://docs.oracle.com/javase/tutorial/rmi/client.html
    * https://docs.oracle.com/javase/tutorial/rmi/example.html
    * http://www.oracle.com/technetwork/java/javase/tech/index-jsp-138781.html
  










# Further references


### RMI in `C#`

  * https://msdn.microsoft.com/en-us/library/ms735119(VS.90).aspx
  * http://www.codemag.com/article/0605051
  
  
  
### HTTP

To get some inspiration on how to implement your own HTTP server, you can read the following article, which gives a corresponding example in Ruby, see https://practicingruby.com/articles/implementing-an-http-file-server.

# Table of Contents

1. [Marshalling and Unmarshalling of XML](#xml_marshalling)
1. [Marshalling and Unmarshalling of JSON](#json_marshalling)
1. [A Simple HTTP Client via TCP](#http_client_tcp)
1. [A Simple HTTP Sever](#http_server)
1. [Demo RPC in Go with Gob Wire Format via TCP](#rpc_gob)
1. [Demo RPC in Go with JSON Wire Format via HTTP](#rpc_http_json)

--------------------

# Marshalling and Unmarshalling of XML <a name="xml_marshalling"></a>


```go
package main

import (
	"encoding/xml"
	"fmt"
	"io/ioutil"
	"log"
	"os"
)

type Post struct {
	XMLName xml.Name `xml:"post"`
	Id      string   `xml:"id,attr"`
	Content string   `xml:"content"`
	Author  Author   `xml:"author"`
}

type Author struct {
	Id   string `xml:"id,attr"`
	Name string `xml:",chardata"`
}

func marshal(post Post, pathToFile string) {
	// output, err := xml.Marshal(&post)
	output, err := xml.MarshalIndent(post, "", "\t")
	if err != nil {
		log.Fatal("Error marshalling to XML:", err)
	}
	err = ioutil.WriteFile(pathToFile, []byte(xml.Header+string(output)), 0644)
	if err != nil {
		log.Fatal("Error writing XML to file:", err)
	}
}

func unmarshal(pathToFile string) Post {
	xmlFile, err := os.Open(pathToFile)
	defer xmlFile.Close()
    if err != nil {
		log.Fatal("Error opening XML file:", err)
	}

	xmlData, err := ioutil.ReadAll(xmlFile)
	if err != nil {
		log.Fatal("Error reading XML data:", err)
	}

	var post Post
	xml.Unmarshal(xmlData, &post)
	fmt.Println(post)

	fmt.Println(post.XMLName.Local)
	fmt.Println(post.Id)
	fmt.Println(post.Content)
	fmt.Println(post.Author)
	fmt.Println(post.Author.Id)
	fmt.Println(post.Author.Name)
	return post
}

func main() {
	post := Post{
		Id:      "1",
		Content: "Hello World!",
		Author: Author{
			Id:   "2",
			Name: "John Doe",
		},
	}

	pathToFile := "../post.xml"
	marshal(post, pathToFile)
	unmarshal(pathToFile)
}
```
--------------------

------------------

# Marshalling and Unmarshalling of JSON <a name="json_marshalling"></a>

```go
package main

import (
	"encoding/json"
	"fmt"
	"io/ioutil"
	"log"
	"os"
)

type Post struct {
	Id       int       `json:"id"`
	Content  string    `json:"content"`
	Author   Author    `json:"author"`
	Comments []Comment `json:"comments"`
}

type Author struct {
	Id   int    `json:"id"`
	Name string `json:"name"`
}

type Comment struct {
	Id      int    `json:"id"`
	Content string `json:"content"`
	Author  string `json:"author"`
}

func marshal(post Post, pathToFile string) {
	output, err := json.MarshalIndent(post, "", "\t")
	if err != nil {
		log.Fatal("Error marshalling to JSON:", err)
	}
	err = ioutil.WriteFile(pathToFile, output, 0644)
	if err != nil {
		log.Fatal("Error writing JSON to file:", err)
	}
}

func unmarshal(pathToFile string) Post {

	jsonFile, err := os.Open(pathToFile)
	defer jsonFile.Close()
    if err != nil {
		log.Fatal("Error opening JSON file:", err)
	}
	jsonData, err := ioutil.ReadAll(jsonFile)
	if err != nil {
		log.Fatal("Error reading JSON data:", err)
	}

	fmt.Println(string(jsonData))
	var post Post
	json.Unmarshal(jsonData, &post)
	fmt.Println(post.Id)
	fmt.Println(post.Content)
	fmt.Println(post.Author.Id)
	fmt.Println(post.Author.Name)
	fmt.Println(post.Comments[0].Id)
	fmt.Println(post.Comments[0].Content)
	fmt.Println(post.Comments[1].Author)

	return post
}

func main() {

	post := Post{
		Id:      1,
		Content: "Hello World!",
		Author: Author{
			Id:   2,
			Name: "John Doe",
		},
		Comments: []Comment{
			{
				Id:      1,
				Content: "Have a great day!",
				Author:  "Adam",
			},
			{
				Id:      2,
				Content: "How are you today?",
				Author:  "Betty",
			},
		},
	}

	pathToFile := "../post.json"
	marshal(post, pathToFile)
	unmarshal(pathToFile)
}
```
------------------

-------------------------


# A Simple HTTP Client via TCP  <a name="http_client_tcp"></a>


```go
package main

import (
	"bufio"
	"fmt"
	"log"
	"net"
	"strings"
)

func main() {
	conn, err := net.Dial("tcp", "lynx.browser.org:80")
	if err != nil {
		log.Panic(err)
	}
	defer conn.Close()

	fmt.Fprintf(conn, "GET / HTTP/1.1\r\n")
	fmt.Fprintf(conn, "Host: lynx.browser.org\r\n")
	// fmt.Fprintf(conn, "User-Agent: curl/7.47.0\r\n")
	fmt.Fprintf(conn, "Accept: */*\r\n\r\n")

	connbuf := bufio.NewReader(conn)
	for {
		str, err := connbuf.ReadString('\n')
		if len(str) > 0 {
			fmt.Println(strings.TrimSpace(str))
		}
		if str == "\r\n" {
			// TODO: Start counting of content length here
            // and break as soon the message is read...
		}
		if err != nil {
			break
		}
	}
}
```

Observer the traffic for example with:

```bash
$ sudo tcpdump -vv -X -A port 80
```

-------------------------

-------------------------


# A Simple HTTP Sever  <a name="http_server"></a>



```go
package main

import (
	"encoding/json"
	"fmt"
	"io/ioutil"
	"log"
	"net/http"
	"os"
)

type Post struct {
	Id      int    `json:"id"`
	Content string `json:"content"`
	Author  Author `json:"author"`
}

type Author struct {
	Id   int    `json:"id"`
	Name string `json:"name"`
}

func marshal(post Post, pathToFile string) {
	output, err := json.MarshalIndent(post, "", "\t")
	if err != nil {
		log.Fatal("Error marshalling to JSON:", err)
	}
	err = ioutil.WriteFile(pathToFile, output, 0644)
	if err != nil {
		log.Fatal("Error writing JSON to file:", err)
	}
}

func unmarshal(pathToFile string) Post {

	jsonFile, err := os.Open(pathToFile)
	if err != nil {
		log.Fatal("Error opening JSON file:", err)
	}
	defer jsonFile.Close()
	jsonData, err := ioutil.ReadAll(jsonFile)
	if err != nil {
		log.Fatal("Error reading JSON data:", err)
	}

	var post Post
	json.Unmarshal(jsonData, &post)
	return post
}

// MyServerApp is an application level context for our service.
type MyServerApp struct{}

// Default is the default greeting.
func (a MyServerApp) Default(res http.ResponseWriter, req *http.Request) {
	// Test for example with:
	/// curl http://192.168.20.2:9080
	fmt.Fprintln(res, "Hello World!")
}

// Post greets requests at the /foo route.
func (a MyServerApp) Post(res http.ResponseWriter, req *http.Request) {
	if req.Method == "GET" {
		// Test for example with: 
		// curl 192.168.20.2:9080/post
		// or more explicitely with: 
		// curl -i -H "Accept: application/json" -H "Content-Type: application/json" -X GET http://192.168.20.2:9080/post
		post := unmarshal("post.json")
		fmt.Fprintln(res, post)
	} else if req.Method == "POST" {
		// Test for example with: 
		// curl -X POST  -H "Accept: Application/json" -H "Content-Type: application/json" http://192.168.20.2:9080/post -d '{"id": 2, "content": "Hello from Helge!", "author": {"id": 3, "name": "Helge"}}'

		body, err := ioutil.ReadAll(req.Body)
		if err != nil {
			log.Println(err.Error())
		}

		var post Post

		err = json.Unmarshal(body, &post)
		if err != nil {
			log.Println(err.Error())
		}
		fmt.Println(post)
		marshal(post, "./received.json")
	}
}

func main() {

	// Create our app
	var a MyServerApp

	// Use http.HandleFunc instead of http.Handle. Instead of requiring a
	// full-blown type implementing http.Handler this just wants any function
	// that accepts a response writer and request.
	http.HandleFunc("/", a.Default)
	http.HandleFunc("/post", a.Post)

	log.Print("Listening on port 9080")
	log.Fatal(http.ListenAndServe(":9080", nil))
}
```


Observer the traffic for example with:

```bash
$ sudo tcpdump -A -i enp0s9 -vv -X dst or src 192.168.20.2 and port 9080
```

---------------------

# Demo RPC in Go with Gob Wire Format via TCP  <a name="rpc_gob"></a>

## Server - TCP with Gob Wire Format

```go
package server

import (
	"errors"
	"fmt"
	"log"
	"net"
	"net/rpc"
	"time"

	"bitbucket.org/HelgeCPH/si_rpc/contract"
)

const port = 1234

func main() {
	StartServer()
}

func StartServer() {
	log.Printf("Server starting on port %v\n", port)

	rpcFuncHandler := &DayRPCHandler{}
	rpc.Register(rpcFuncHandler)

	l, err := net.Listen("tcp", fmt.Sprintf(":%v", port))
	if err != nil {
		log.Fatal(fmt.Sprintf("Unable to listen on given port: %s", err))
	}
	defer l.Close()

	for {
		conn, _ := l.Accept()
		go rpc.ServeConn(conn)
	}
}

type DayRPCHandler struct{}

func (h *DayRPCHandler) GetDay(args *contract.DayRPCRequest, reply *contract.DayRPCResponse) error {

	daytime := time.Now()

	switch args.Qualifier {
	case "Yesterday":
		reply.Day = daytime.Add(-24 * time.Hour).Weekday().String()
	case "Today":
		reply.Day = daytime.Weekday().String()
	case "Tomorrow":
		reply.Day = daytime.Add(24 * time.Hour).Weekday().String()
	default:
		log.Print("Unrecognized qualifier")
		return errors.New("Unrecognized qualifier")
	}

	return nil
}
```


## Client - TCP with Gob Wire Format

```go
package client

import (
	"fmt"
	"log"
	"net/rpc"

	"bitbucket.org/HelgeCPH/si_rpc/contract"
)

const port = 1234

func CreateClient() *rpc.Client {
	client, err := rpc.Dial("tcp", fmt.Sprintf("192.168.20.2:%v", port))
	if err != nil {
		log.Fatal("Cannot call server:", err)
	}

	return client
}

func PerformRequest(client *rpc.Client) contract.DayRPCResponse {
	args := &contract.DayRPCRequest{Qualifier: "Yesterday"}
	var reply contract.DayRPCResponse

	err := client.Call("DayRPCHandler.GetDay", args, &reply)
	if err != nil {
		log.Fatal("error:", err)
	}

	return reply
}
```

## Contract and Upstart

```go
package contract

type DayRPCRequest struct {
	Qualifier string
}

type DayRPCResponse struct {
	Day string
}
```

```go
package main

import (
	"fmt"
	"log"
	"os/exec"
	"strings"

	"bitbucket.org/HelgeCPH/si_rpc/client"
	"bitbucket.org/HelgeCPH/si_rpc/server"
)

func getHostName() string {

	out, err := exec.Command("hostname").Output()
	if err != nil {
		log.Fatal(err)
	}
	return strings.Trim(string(out), "\n")
}

func main() {

	if getHostName() == "godevnode1" {
		server.StartServer()
	} else {
		c := client.CreateClient()
		defer c.Close()

		reply := client.PerformRequest(c)
		fmt.Println(reply.Day)
	}
}
```

Observer the traffic for example with:

~~~bash
$ sudo tcpdump -A -i enp0s9 -vv -X dst or src 192.168.20.2 and port 1234
~~~
-------------------------

---------------------
# Demo RPC in Go with JSON Wire Format via HTTP  <a name="rpc_http_json"></a>

## Server - HTTP with JSON Wire Format

```go
package server

import (
	"errors"
	"fmt"
	"io"
	"log"
	"net"
	"net/http"
	"net/rpc"
	"net/rpc/jsonrpc"
	"time"

	"bitbucket.org/HelgeCPH/si_rpc_http_json/contract"
)

const port = 1234

type HttpConn struct {
	in  io.Reader
	out io.Writer
}

func (c *HttpConn) Read(p []byte) (n int, err error)  { return c.in.Read(p) }
func (c *HttpConn) Write(d []byte) (n int, err error) { return c.out.Write(d) }
func (c *HttpConn) Close() error                      { return nil }

type DayRPCHandler struct{}

func (h *DayRPCHandler) GetDay(args *contract.DayRPCRequest, reply *contract.DayRPCResponse) error {

	fmt.Println(args)
	daytime := time.Now()

	switch args.Qualifier {
	case "Yesterday":
		reply.Day = daytime.Add(-24 * time.Hour).Weekday().String()
	case "Today":
		reply.Day = daytime.Weekday().String()
	case "Tomorrow":
		reply.Day = daytime.Add(24 * time.Hour).Weekday().String()
	default:
		log.Print("Unrecognized qualifier")
		return errors.New("Unrecognized qualifier")
	}

	return nil
}

func StartServer() {
	rpc.Register(&DayRPCHandler{})

	l, err := net.Listen("tcp", fmt.Sprintf(":%v", port))
	if err != nil {
		log.Fatal(fmt.Sprintf("Unable to listen on given port: %s", err))
	}

	http.Serve(l, http.HandlerFunc(httpHandler))
}

func httpHandler(w http.ResponseWriter, r *http.Request) {
	serverCodec := jsonrpc.NewServerCodec(&HttpConn{in: r.Body, out: w})
	err := rpc.ServeRequest(serverCodec)

	if err != nil {
		log.Printf("Error while serving JSON request: %v", err)
		http.Error(w, "Error while serving JSON request, details have been logged.", 500)
		return
	}
}
```


## Client - HTTP with JSON Wire Format

```go
package client

import (
	"bytes"
	"encoding/json"
	"fmt"
	"io/ioutil"
	"log"
	"net/http"
	"net/rpc"

	"bitbucket.org/HelgeCPH/si_rpc_http_json/contract"
)

func PerformRequest() contract.DayRPCResponse {
	// similar to:
	// curl -X POST  -H "Accept: Application/json" -H "Content-Type: application/json" http://192.168.20.2:1234/ -d '{"id": 1, "method": "DayRPCHandler.GetDay", "params": [{"qualifier":"Yesterday"}]}'
	r, _ := http.Post(
		"http://192.168.20.2:1234",
		"application/json",
		bytes.NewBuffer([]byte(`{"id": 1, "method": "DayRPCHandler.GetDay", "params": [{"qualifier":"Today"}]}`)),
	)
	defer r.Body.Close()

	body, _ := ioutil.ReadAll(r.Body)

	var response map[string]interface{}
	json.Unmarshal([]byte(body), &response)

	result, _ := response["result"].(map[string]interface{})
	fmt.Println(result["Day"])

	return contract.DayRPCResponse{Day: string(result["Day"].(string))}
}

// rpcRequest represent a RCP request
type rpcRequest struct {
	Method string      `json:"method"`
	Params interface{} `json:"params"`
	Id     int64       `json:"id"`
}

type rpcResponse struct {
	Id     int64           `json:"id"`
	Result json.RawMessage `json:"result"`
	Err    interface{}     `json:"error"`
}

func PerformRequest3() contract.DayRPCResponse {
	args := contract.DayRPCRequest{Qualifier: "Today"}
	var args_list []interface{}
	args_list = append(args_list, args)
	rpcR := rpcRequest{"DayRPCHandler.GetDay", args_list, 1}

	output, err := json.Marshal(rpcR)
	if err != nil {
		log.Fatal("Error marshalling to JSON:", err)
	}

	var reply contract.DayRPCResponse
	var response rpcResponse
	r, _ := http.Post("http://192.168.20.2:1234", "application/json", bytes.NewBuffer(output))

	jsonData, err := ioutil.ReadAll(r.Body)
	if err != nil {
		log.Fatal("Error reading JSON data:", err)
	}
	json.Unmarshal(jsonData, &response)
	json.Unmarshal(response.Result, &reply)

	return reply
}
```

## Contract and Upstart

```go
package contract

type DayRPCRequest struct {
	Qualifier string `json:"qualifier"`
}

type DayRPCResponse struct {
	Day string `json:"Day"`
}
```

```go
package main

import (
	"fmt"
	"log"
	"os/exec"
	"strings"

	"bitbucket.org/HelgeCPH/si_rpc_http_json/client"
	"bitbucket.org/HelgeCPH/si_rpc_http_json/server"
)

func getHostName() string {

	out, err := exec.Command("hostname").Output()
	if err != nil {
		log.Fatal(err)
	}
	return strings.Trim(string(out), "\n")
}

func main() {

	if getHostName() == "godevnode1" {
		server.StartServer()
	} else {
		// curl -X POST  -H "Accept: Application/json" -H "Content-Type: application/json" http://192.168.20.2:1234/ -d '{"id": 1, "method": "DayRPCHandler.GetDay", "params": [{"qualifier":"Today"}]}'
		reply := client.PerformRequest3()
		fmt.Println(reply.Day)
	}
}
```


Observer the traffic for example with:

~~~bash
$ sudo tcpdump -A -i enp0s9 -vv -X dst or src 192.168.20.2 and port 1234
~~~

------------------------- 

# Your turn!

![](images/your_turn.gif)

## First Hour

  1. Your own HTTP server
    * Write your own HTTP server, which implements a the HTTP verbs `GET` and `POST` and can handle some information in headers, such as which document types are supported. Do that in the language of your choice.
    * Observe the traffic to and from your server via TCPDump. 
      - Can you replace the webserver from task 1 above with your own webserver?
      - Do the HTTP packages on the wire look the same as before?
      - Again, what makes an HTTP response an HTTP response?   
  2. A Proper HTTP Server
    * In the language of your choice, create an HTTP server and let it serve documents in various formats (XML and JSON) -on a `GET`- and let it receive documents -on a `POST`.
    * If you need inspiration, make use of the small test program, which you were given in the LSD class. Otherwise use `curl` or `wget` as a client program, as in the examples above.    
    * Build and deploy your server on one node of your small virtual network, see last lecture, and run the client from another node.
    * Observe the traffic to and from your server via TCPDump. 
      - What can you see? What makes an HTTP request an HTTP request?
      - What makes an HTTP response an HTTP response?
      - How does your traffic look like for larger documents?
 
  
## Second and Third Hour

  1. RPCs and RMIs  
    
    * In the language of your choice, write a set of at least two distributed programs that call each other's functions or methods. In the best case, if you have time enough write RMIs in Java and RPCs in another language.
    * Observe the traffic to and from your server via TCPDump.
    * What can you see? 
      - Which protocol is used by your framework?
      - What is the wire format of the exchanged data?

  

## Reflection

  * What precisely is the difference between RPCs and RMIs? What are the practical consequences of these differences?
  * What protocol use RPCs/RMIs for information exchange in your language/framework?
  * What means marshalling and unmashalling in the realm of Object-orientation? In particular, what does it mean to marshal an object?
  * What are typical characteristics of RPC/RMI frameworks with respect to the languages used in the various sub-systems?
  
  
![](images/fig06_07.jpg)  
![](images/fig06_08.jpg)  